## imports

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,742 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,019 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.0/810.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: IPython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.21.0 which is incompatible.


## useful stuff

In [ ]:
import PIL
from manim import *
class Node(Circle):
    def __init__(self, radius=0.05, color=YELLOW, fill_opacity=1, **kwargs):
        super().__init__(radius=radius, color=color, fill_opacity=fill_opacity, **kwargs)


class Resistor(VGroup):
    def __init__(self, length=1, width=0.5, num_lines=5, **kwargs):
        super().__init__(**kwargs)

        # Create the zigzag lines
        lines = VGroup()
        delta_x = length / num_lines
        delta_y = width / 2
        for i in range(num_lines):
            start = i * delta_x * RIGHT + delta_y * UP
            end = (i + 1) * delta_x * RIGHT
            line = Line(start, end)
            if i % 2 == 1:
                line.flip()
            lines.add(line)

        # Add the lines to the Resistor object
        self.add(lines)

        # Create nodes at the ends of the resistor
        node1 = Node().move_to(lines[0].get_start())
        node2 = Node().move_to(lines[-1].get_end())

        # Add nodes to the Resistor object
        self.nodes = VGroup(node1,node2)
        self.add(self.nodes)


class JosephsonJunction(VGroup):
    def __init__(self, width=1, height=1, line_width=2, **kwargs):
        super().__init__(**kwargs)

        # Create the box
        box = Rectangle(width=width, height=height, fill_opacity=0, stroke_opacity=1)
        self.add(box)

        # Calculate the coordinates for the lines
        line1_start = box.get_corner(UL)
        line1_end = box.get_corner(DR)
        line2_start = box.get_corner(UR)
        line2_end = box.get_corner(DL)

        # Create the lines
        line1 = Line(line1_start, line1_end, stroke_width=line_width, color="WHITE")
        line2 = Line(line2_start, line2_end, stroke_width=line_width, color="WHITE")
        lines = VGroup(line1, line2)
        self.add(lines)

        # Create nodes at the edges of the junction
        node1 = Node().move_to((line1_start + line2_start) / 2)
        node2 = Node().move_to((line2_end + line1_end) / 2)
        self.nodes = VGroup(node1,node2)
        self.add(self.nodes)


class Capacitor(VGroup):
    def __init__(self, height=1, width=0.1, gap=0.1, rotation=PI/2, **kwargs):
        super().__init__(**kwargs)

        # Create the two parallel lines
        line1 = Line(UP * height / 2, DOWN * height / 2, stroke_width=DEFAULT_STROKE_WIDTH)
        line2 = line1.copy()

        # Set the position of the lines
        line1.shift(LEFT * (width / 2 + gap / 2))
        line2.shift(RIGHT * (width / 2 + gap / 2))

        # Add the lines to the Capacitor object
        self.add(line1, line2).rotate(rotation)

        # Create nodes at the ends of the capacitor
        node1 = Node().move_to((line1.get_start() + line1.get_end()) / 2)
        node2 = Node().move_to((line2.get_start() + line2.get_end()) / 2 )
        self.nodes = VGroup(node1,node2)
        self.add(self.nodes)


class Inductor(VGroup):
    def __init__(self, radius=0.5, num_segments=3, width=2, **kwargs):
        super().__init__(**kwargs)

        # Calculate the angle per segment
        angle_per_segment = PI / num_segments

        # Create the semicircles
        semicircles = VGroup()
        for i in range(num_segments):
            start_angle = i * angle_per_segment
            end_angle = start_angle + angle_per_segment
            semicircle = Arc(radius=radius/3, start_angle=0, angle=PI, stroke_width=width).rotate(PI)
            semicircles.add(semicircle)

        # Add the semicircles to the Inductor object
        semicircles.arrange(direction=RIGHT, buff=0)
        self.add(semicircles).rotate(-1*PI/2)

        # Create nodes at the ends of the inductor
        start_node = Node().move_to(semicircles[0].get_start())
        end_node = Node().move_to(semicircles[-1].get_end())
        self.nodes = VGroup(start_node, end_node)
        self.add(self.nodes)



class Wire(Line):
    def __init__(self, node1, node2, **kwargs):
        super().__init__(node1.get_center(), node2.get_center(), **kwargs)


class ReferenceGrid(VGroup):
    def __init__(self, width=10, height=10, grid_spacing=1, **kwargs):
        super().__init__(**kwargs)

        # Create the reference grid
        x_ticks = np.arange(-1*width - grid_spacing , width + grid_spacing, grid_spacing)
        y_ticks = np.arange(-1*width - grid_spacing, height + grid_spacing, grid_spacing)

        # Create dots at integer coordinates
        dots = VGroup(*[
            Dot(color=LIGHT_GREY).scale(0.5).move_to([x, y, 0]).set_opacity(.2)
            for x in x_ticks
            for y in y_ticks
        ])
        self.add(dots)

        # Mark the origin with a blue dot
        origin_dot = Dot(color=BLUE).scale(0.2)
        self.add(origin_dot)


class CooperPairBox(VGroup):
    def __init__(self):
        super().__init__()

        # Create the capacitor and Josephson junction objects
        capacitor = Capacitor().shift(LEFT)
        junction = JosephsonJunction().shift(RIGHT)
        capacitor_ground = Capacitor().shift(2 * UP)
        upper_left = Node().shift(LEFT).shift(UP)
        upper_right = Node().shift(RIGHT).shift(UP)
        lower_left = Node().shift(LEFT).shift(DOWN)
        lower_right = Node().shift(RIGHT).shift(DOWN)
        upper_mid = Node(fill_opacity=0).shift(UP)
        lower_mid = Node().shift(DOWN)
        upper_upper_mid = Node().shift(3 * UP)
        self.add(capacitor, junction, capacitor_ground)


        # Create the nodes for the capacitor and junction
        cap_node1, cap_node2 = capacitor.nodes
        cap_node3, cap_node4 = capacitor_ground.nodes
        jj_node1, jj_node2 = junction.nodes

        self.nodes = VGroup(
            upper_left, upper_right, lower_left, lower_right, upper_mid, lower_mid,
            upper_upper_mid, cap_node1, cap_node2, cap_node3, cap_node4, jj_node1, jj_node2
        )
        self.add(self.nodes)

        # Create the wire connecting the capacitor and junction
        wire_1 = Wire(upper_left, cap_node2)
        wire_2 = Wire(cap_node1, lower_left)
        wire_3 = Wire(lower_left, lower_right)
        wire_4 = Wire(lower_right, jj_node2)
        wire_5 = Wire(jj_node1, upper_right)
        wire_6 = Wire(upper_right, upper_left)
        wire_7 = Wire(upper_mid, cap_node3)
        wire_8 = Wire(cap_node4, upper_upper_mid)

        wires = VGroup(wire_1, wire_2, wire_3, wire_4, wire_5, wire_6, wire_7, wire_8)
        self.add(wires)

        # Create the labels
        Cap_label = Text("C").next_to(capacitor).scale(0.5).shift(2 * LEFT)
        Cap_to_gnd = Text("Cg").next_to(capacitor_ground).scale(0.5)
        JJ_label = Text("JJ").next_to(junction).scale(0.5)


        # Create a VGroup to hold all the elements
        CPB = VGroup(capacitor, capacitor_ground, junction, wires, Cap_label, Cap_to_gnd, JJ_label)
        self.add(CPB)


class resonator(VGroup):
    def __init__(self):
        super().__init__()

        # Create the capacitor and Josephson junction objects
        capacitor = Capacitor().shift(LEFT)
        inductor = Inductor().shift(RIGHT)
        capacitor_ground = Capacitor().shift(2 * UP)
        upper_left = Node().shift(LEFT).shift(UP)
        upper_right = Node().shift(RIGHT).shift(UP)
        lower_left = Node().shift(LEFT).shift(DOWN)
        lower_right = Node().shift(RIGHT).shift(DOWN)
        upper_mid = Node(fill_opacity=0).shift(UP)
        lower_mid = Node().shift(DOWN)
        upper_upper_mid = Node().shift(3 * UP)
        self.add(capacitor, inductor, capacitor_ground)


        # Create the nodes for the capacitor and junction
        cap_node1, cap_node2 = capacitor.nodes
        cap_node3, cap_node4 = capacitor_ground.nodes
        i_node1, i_node2 = inductor.nodes

        self.nodes = VGroup(
            upper_left, upper_right, lower_left, lower_right, upper_mid, lower_mid,
            upper_upper_mid, cap_node1, cap_node2, cap_node3, cap_node4, i_node1, i_node2
        )
        self.add(self.nodes)

        # Create the wire connecting the capacitor and junction
        wire_1 = Wire(upper_left, cap_node2)
        wire_2 = Wire(cap_node1, lower_left)
        wire_3 = Wire(lower_left, lower_right)
        wire_4 = Wire(lower_right, i_node2)
        wire_5 = Wire(i_node1, upper_right)
        wire_6 = Wire(upper_right, upper_left)
        wire_7 = Wire(upper_mid, cap_node3)
        wire_8 = Wire(cap_node4, upper_upper_mid)

        wires = VGroup(wire_1, wire_2, wire_3, wire_4, wire_5, wire_6, wire_7, wire_8)
        self.add(wires)

        # Create the labels
        Cap_label = Text("C").next_to(capacitor).scale(0.5).shift(2 * LEFT)
        Cap_to_gnd = Text("Cg").next_to(capacitor_ground).scale(0.5)
        L_label = Text("L").next_to(inductor).scale(0.5)


        # Create a VGroup to hold all the elements
        CPW = VGroup(capacitor, capacitor_ground, inductor, wires, Cap_label, Cap_to_gnd, L_label)
        self.add(CPW)


class Ground(VGroup):
    def __init__(self, length=.8, wire_length=0.2, wire_angle=PI / 2, **kwargs):
        super().__init__(**kwargs)

        # Create the parallel lines
        line_1 = Line(RIGHT * length / 2).move_to(ORIGIN)
        line_2 = Line(RIGHT * length / 4).move_to(ORIGIN).shift(UP*0.2)
        line_3 = Line(RIGHT * length / 8).move_to(ORIGIN).shift(UP*0.4)

        lines = VGroup(line_3, line_2, line_1)



        # Create the node at the top of the perpendicular wire
        node_1 = Node().move_to(line_3.get_center())
        node_2 = Node().move_to(node_1).shift(UP)

        wire = Wire(node_1, node_2)

        # Create the ground symbol
        ground_symbol = VGroup(lines,wire)

        # Add the ground symbol to the Ground object
        self.add(ground_symbol)


        # Add the node to the Ground object
        self.nodes = VGroup(node_1, node_2)
        self.add(self.nodes)


class TransistorSymbol(VGroup):
    def __init__(self, size=1, color=WHITE, **kwargs):
        super().__init__(**kwargs)

        node_1 = Node().shift(LEFT*0.5)  # leftmost node
        node_2 = Node()  # center node
        node_3 = Node().shift(UP*.5)  # top of T
        node_4 = Node().shift(DOWN*0.5)  # bottom of T
        node_5 = Node().move_to(node_3).shift(UP*0.5).shift(RIGHT*0.5)
        node_6 = Node().move_to(node_5).shift(UP*0.5)
        node_7 = Node().move_to(node_4).shift(DOWN*0.5).shift(RIGHT*0.5)
        node_8 = Node().move_to(node_7).shift(DOWN*0.5)

        wire_1 = Wire(node_1, node_2)
        wire_2 = Wire(node_2, node_3)
        wire_3 = Wire(node_2, node_4)
        wire_4 = Wire(node_3, node_5)
        wire_5 = Wire(node_5, node_6)
        wire_6 = Wire(node_4, node_7)
        wire_7 = Wire(node_7, node_8)

        # Create a VGroup to hold all the nodes and wires
        self.nodes = VGroup(node_1, node_2, node_3, node_4, node_5, node_6, node_7, node_8)
        self.wires = VGroup(wire_1, wire_2, wire_3, wire_4, wire_5, wire_6, wire_7)

        # Add the nodes to the TransistorSymbol object
        self.add(self.nodes)
        self.add(self.wires)

class SwitchSymbol(VGroup):
    def __init__(self, size=1, color=WHITE, **kwargs):
        super().__init__(**kwargs)
        self.size = size  # Store size for scaling

        # Define nodes for the switch
        self.node_1 = Node().shift(LEFT * 0.5)  # Leftmost node
        self.node_2 = Node()  # End node for wire1, closed position for wire2
        self.node_3 = Node().shift(RIGHT * 0.1, UP * 0.5)  # Open position for wire2
        self.node_4 = Node().shift(RIGHT * 0.5)  # Origin node for wire2, wire3
        self.node_5 = Node().shift(RIGHT * 1)  # End node for wire3

        # Define wires for the open/closed switch
        self.wire_1 = Wire(self.node_1, self.node_2)  # Leftmost wire
        self.wire_2_open = Wire(self.node_4, self.node_3)  # Wire in open position
        self.wire_2_closed = Wire(self.node_4, self.node_2)  # Wire in closed position
        self.wire_3 = Wire(self.node_4, self.node_5)  # Wire leading to node 5

        # Group nodes and wires
        self.nodes = VGroup(self.node_1, self.node_2, self.node_3, self.node_4, self.node_5)
        self.switch_open = VGroup(self.wire_1, self.wire_2_open, self.wire_3)
        self.switch_closed = VGroup(self.wire_1, self.wire_2_closed, self.wire_3)

        # Initial state (open)
        self.add(self.nodes, self.switch_open)

        # Ensure all wires are updated dynamically
        self.wire_2_closed.move_to(self.wire_2_open)

    def sclose_switch(self):
        """Transition to closed state by dynamically updating wire positions."""
        # Create a dynamically updated wire for the closed position
        updated_wire_2_closed = Wire(
            self.node_4,
            self.node_2,
            color=self.wire_2_closed.get_color()
        )

        # Animation to replace the open wire with the closed wire
        return AnimationGroup(
            FadeOut(self.wire_2_open),  # Remove the open wire
            Create(updated_wire_2_closed),  # Add the closed wire
            lag_ratio=0
        )


    def scale(self, scale_factor, **kwargs):
        """Override scale to ensure nodes and wires scale correctly."""
        super().scale(scale_factor, **kwargs)
        self.size *= scale_factor  # Update internal size tracking
        return self

    def move_to(self, position, **kwargs):
        """Override move_to to ensure all components move together."""
        super().move_to(position, **kwargs)
        return self

    def shift(self, direction, **kwargs):
        """Override shift to ensure all components shift correctly."""
        super().shift(direction, **kwargs)
        return self

class Transmon(VGroup):
  def __init__(self,isgrid=1, **kwargs):
    super().__init__(**kwargs)
    CPB = CooperPairBox().shift(RIGHT*2.5)
    CPW = resonator().shift(LEFT*2.5)
    Cap = Capacitor().shift(DOWN*1.6).rotate(PI/2)
    grid = ReferenceGrid()
    if isgrid==0:
      self.add(grid)

    ground = Ground().shift(DOWN*3,RIGHT*2.5)
    ground2 = Ground().shift(DOWN*3,LEFT*2.5)

    wire = Wire(CPB.nodes[5],ground.nodes[1])
    wire2 = Wire(CPW.nodes[5],ground2.nodes[1])
    wire3 = Wire(ground.nodes[1],Cap.nodes[0])
    wire4 = Wire(ground2.nodes[1],Cap.nodes[1])
    wires = VGroup(wire,wire2,wire3,wire4)

    Cap2 = Cap.copy().move_to(ground2.nodes[1].get_center()).shift(2*LEFT)
    wire5 = Wire(ground.nodes[1],Cap2.nodes[0])
    node = Node().move_to(Cap2.nodes[1].get_center()+LEFT)
    wire6 = Wire(Cap2.nodes[1],node)
    wires.add(wire5)
    wires.add(wire6)

    self.qubit = VGroup(CPW, CPB,Cap,wires,Cap2)
    self.add(self.qubit)

######----------------------------------quantum circuits
from manim import *

class Hadamard(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      hadamard_gate_sq = Square(.625, color=BLUE, fill_opacity=.9)
      hadamard_text = Text("H").move_to(hadamard_gate_sq.get_center())
      self.hadamard_gate = VGroup(hadamard_gate_sq,hadamard_text)
      self.add(self.hadamard_gate)

class Oracle(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      gate_sq = Rectangle(height=3, width = 1, color=BLUE, fill_opacity=.9)
      gate_text = MathTex("U_f").move_to(gate_sq.get_center())
      self.gate = VGroup(gate_sq, gate_text)
      self.add(self.gate)

class Not(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      x_gate_sq = Square(.625, color=PURPLE, fill_opacity=.9)
      x_text = Text("X").move_to(x_gate_sq.get_center())
      self.x_gate = VGroup(x_gate_sq,x_text)
      self.add(self.x_gate)

class Measurement(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      m_sq = Square(.625, color=RED, fill_opacity=.9)
      m_text = Text("M").move_to(m_sq.get_center())
      self.m_gate = VGroup(m_sq,m_text)
      self.add(self.m_gate)

class Identity(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      I_sq = Square(.625, color=ORANGE, fill_opacity=.9)
      I_text = Text("I").move_to(I_sq.get_center())
      self.I_gate = VGroup(I_sq,I_text)
      self.add(self.I_gate)


class Zgate(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      z_gate_sq = Square(.625, color=RED, fill_opacity=.9)
      z_text = Text("Z").move_to(z_gate_sq.get_center())
      self.z_gate = VGroup(z_gate_sq,z_text)
      self.add(self.z_gate)


class Cnot(VGroup):
  def __init__(self,**kwargs):
      super().__init__(**kwargs)
      col = GREEN
      dot = Dot(color = col)
      box = Square(side_length = 1, color = col, fill_opacity=.9).next_to(dot.get_center(),DOWN,buff=1)
      line = Line(dot.get_center(),box.get_top(),color=col)
      text = Text("CNOT").move_to(box.get_center()).scale(.5)
      self.cnot_gate = VGroup(dot, box, line, text)
      self.add(self.cnot_gate)


class Register(VGroup):
  #make a register of qubits to do operations on
  def __init__(self, n, length, **kwargs):
      super().__init__(**kwargs)
      #make a bunch of qubits
      self.qubits = VGroup()
      for i in range(n):
        qubit = MathTex(r"|0\rangle") #.shift(4*LEFT, 2*UP + i*.7*DOWN)
        self.qubits.add(qubit)

      self.qubits.arrange(DOWN, buff=1)

      if n>3:
        #make ...
        self.dots =  VGroup()
        for i in range(3):
          dot_ = Dot().next_to(self.qubits[-1],DOWN,buff=.25).shift(DOWN*i*.5)
          self.dots.add(dot_)

        #make last qubit
        qubit_final = MathTex(r"|0\rangle").move_to(self.dots[-1]).shift(.5*DOWN)
        self.qubits.add(qubit_final)

      elif n<=3:
        self.dots = VGroup()

      #make lines going from the qubits to the right
      self.lines = VGroup()
      for qubit in self.qubits:
        line = Line(qubit.get_right(),qubit.get_right()+length*RIGHT)
        self.lines.add(line)
      self.register = VGroup(self.lines, self.dots,self.qubits)
      self.add(self.register)
      self.add(self.lines)


######---------------------------------- Neutral Atoms


class GeneralAtom(VGroup):
    def __init__(self, size=0.5, element="H", electron_configuration=None, VERBOSE=False, **kwargs):
        super().__init__(**kwargs)
        self.size = size
        self.element = element
        self.electron_configuration = electron_configuration if electron_configuration else []

        # Set up nucleus
        self.nucleus = Dot(radius=1 * self.size, color=PURPLE)
        if VERBOSE:
          nucleus_label = MathTex(element, color=WHITE).scale(.75).move_to(self.nucleus.get_center())
        else:
          nucleus_label = MathTex('')
        # Add nucleus and label
        self.add(self.nucleus, nucleus_label)

        # Set up orbitals
        self.orbitals = VGroup()
        self.electrons = VGroup()

        for n, num_electrons in enumerate(self.electron_configuration, start=1):
            orbital = Circle(radius=2 * self.size * n, color=WHITE)
            self.orbitals.add(orbital)

            for i in range(num_electrons):
                angle = i * (2 * PI / num_electrons)
                electron = Dot(radius=.5 * self.size, color=BLUE)
                electron.move_to(orbital.point_at_angle(angle))
                self.electrons.add(electron)

        if VERBOSE:
            orbital_labels = VGroup(
                *[
                    MathTex(f"n={n}", color=WHITE).next_to(orbital, LEFT).scale(0.5)
                    for n, orbital in enumerate(self.orbitals, start=1)
                ]
            )
            self.add(self.orbitals, self.electrons, orbital_labels)
        else:
            self.add(self.orbitals, self.electrons)

    def animate_electrons(self, scene, rotations=1, run_time=4):
        """Animate electrons orbiting the nucleus."""
        animations = []
        for n, orbital in enumerate(self.orbitals):
            for electron in self.electrons[n::len(self.orbitals)]:
                animations.append(Rotate(electron, angle=2 * PI * rotations, about_point=self.nucleus.get_center(), run_time=run_time, rate_func=linear))
        scene.play(*animations)

class NeutralAtom(VGroup):
    def __init__(self, size=0.5, state=0, VERBOSE=False, **kwargs):
        super().__init__(**kwargs)
        self.size = size
        self.state = state

        # Set up the Bohr model atom
        self.electron = Dot(radius=0.6 * self.size, color=BLUE)
        self.nucleus = Dot(radius=0.3 * self.size, color=PURPLE)
        self.orbit1 = Circle(radius=2 * self.size, color=WHITE)
        self.orbit2 = Circle(radius=4 * self.size, color=WHITE)

        if state == 0:
            self.electron.move_to(self.orbit1.get_left())
        elif state == 1:
            self.electron.move_to(self.orbit2.get_left()).set_color(RED)

        # Add labels
        electron_label = MathTex("e^{-}", color=BLUE).next_to(self.electron, UP)
        nucleus_label = MathTex("N", color=PURPLE).next_to(self.nucleus, DOWN)
        orbit1_label = MathTex("n=1", color=WHITE).next_to(self.orbit1, LEFT).shift(0.1 * RIGHT)
        orbit2_label = MathTex("n=2", color=WHITE).next_to(self.orbit2, LEFT).shift(0.1 * RIGHT)
        labels_group = VGroup(electron_label, nucleus_label, orbit1_label, orbit2_label)

        # Add everything to the atom group
        if VERBOSE:
            atom_group = VGroup(self.orbit1, self.orbit2, self.electron, self.nucleus, labels_group).scale(self.size)
        else:
            atom_group = VGroup(self.orbit1, self.orbit2, self.electron, self.nucleus).scale(self.size)

        self.add(atom_group)

    def change_state(self, new_state):
        """Change the state of the atom and update its visual representation."""
        self.state = new_state
        self.clear_superposition()
        if new_state == 0:
            self.electron.move_to(self.orbit1.get_left()).set_color(BLUE)
        elif new_state == 1:
            self.electron.move_to(self.orbit2.get_left()).set_color(RED)

    def set_superposition(self):
        """Set the atom to a superposition state visually."""
        self.state = 'superposition'
        self.electron.set_opacity(0)  # Hide the main electron
        self.superposition_electrons = VGroup(
            Dot(radius=0.6 * self.size, color=BLUE).move_to(self.orbit1.get_left()).set_opacity(0.5),
            Dot(radius=0.6 * self.size, color=RED).move_to(self.orbit2.get_left()).set_opacity(0.5)
        )
        self.add(self.superposition_electrons)

    def clear_superposition(self):
        """Clear the superposition state, restoring the atom to a single state."""
        if hasattr(self, 'superposition_electrons'):
            self.remove(self.superposition_electrons)
            del self.superposition_electrons


    def start_rotation(self, scene, rotations=1, run_time=4):
        """Start the electron orbiting the nucleus.

        Args:
            scene: The Manim scene to animate in.
            rotations: Number of complete rotations the electron should make.
            run_time: Duration of the animation in seconds.
        """
        if self.state == 0:
            orbit = self.orbit1
        elif self.state == 1:
            orbit = self.orbit2
        else:
            return  # No rotation for superposition or undefined state

        # Calculate the total angle of rotation based on the number of rotations
        total_angle = 2 * PI * rotations

        # Animate the electron orbiting the nucleus
        scene.play(
            Rotate(self.electron, angle=total_angle, about_point=self.nucleus.get_center(), run_time=run_time),
            rate_func=linear
        )

class RydbergAtom(VGroup):
  def __init__(self, size=.5,state=1, VERBOSE=False, **kwargs,):
        super().__init__(**kwargs)
        # Set up the Bohr model atom
        self.size = size
        electron = Dot(radius=0.2, color=BLUE)
        nucleus = Dot(radius=0.15, color=PURPLE)
        orbit1 = Circle(radius=1, color=WHITE)
        orbit2 = Circle(radius=6, color=WHITE)
        if state == 0:
          electron.move_to(orbit1.get_left())
        elif state == 1:
          electron.move_to(orbit2.get_left())


        # Add labels
        electron_label = MathTex("e^{-}", color=BLUE).next_to(electron, UP)
        nucleus_label = MathTex("N", color=PURPLE).next_to(nucleus, DOWN)
        orbit1_label = MathTex("n=1", color=WHITE).next_to(orbit1, LEFT).shift(0.1 * RIGHT)
        orbit2_label = MathTex("n=big", color=WHITE).next_to(orbit2, LEFT).shift(0.1 * RIGHT)
        labels_group = VGroup(electron_label, nucleus_label, orbit1_label, orbit2_label)
        # Add everything to the atom group
        if VERBOSE:
          atom_group = VGroup(orbit1, orbit2, electron, nucleus, labels_group).scale(self.size)
        else:
          atom_group = VGroup(orbit1, orbit2, electron, nucleus).scale(self.size)

        self.add(atom_group)


###-computational complexity-----------------------------------------------------------------------------------------------------

class ComplexityComparison(Scene):
    def __init__(self, complexities, x_max=10000, y_max=10000, labels=None, step=100, **kwargs):
        """
        :param complexities: List of lambda functions representing complexity curves.
                             Example: [lambda x: np.log(x + 1), lambda x: x ** 2]
        :param x_max: Maximum value of x to plot
        :param y_max: Maximum value of y to plot
        :param labels: List of strings for the labels of each curve.
                       If None, labels will be autogenerated as "Complexity 1", "Complexity 2", etc.
        :param step: Step size for sampling x-values (affects resolution)

        Example usage:
          class ComplexityExample(ComplexityComparison):
           def __init__(self, **kwargs):
              super().__init__(
                  complexities=[
                      lambda x: 1,                       # O(1)
                      lambda x: np.log(x + 1),          # O(log(n))
                      lambda x: x,                      # O(n)
                      lambda x: x ** 2 / 1000,          # O(n^2), scaled
                      lambda x: 2 ** (x/100),         # O(2^n), scaled
                  ],
                  labels=[
                      "O(1)",
                      "O(log(n))",
                      "O(n)",
                      "O(n^2)",
                      "O(2^n)"
                  ],
                  x_max=10000,
                  y_max=20000,
                  **kwargs
            )

        """
        super().__init__(**kwargs)
        self.complexities = complexities
        self.x_max = x_max
        self.y_max = y_max
        self.labels = labels or [f"Complexity {i + 1}" for i in range(len(complexities))]
        self.colors = [YELLOW, GREEN, RED, BLUE, PURPLE]  # Predefined color palette
        self.step = step

    def construct(self):
        k = ValueTracker(1)  # Start at a small non-zero value
        axes = Axes(
            x_range=[0, self.x_max, self.x_max // 4],
            y_range=[0, self.y_max, self.y_max // 5],
            x_length=6,
            y_length=6,
            axis_config={"tip_shape": StealthTip, "font_size": 20},
        ).add_coordinates()

        self.play(Create(axes))
        graphs = VGroup()
        labels_group = VGroup()

        # Precompute valid x-values for each complexity
        precomputed_x_ranges = []
        for complexity in self.complexities:
            x_vals = np.arange(0.1, self.x_max, self.step)
            valid_x = x_vals[np.array([complexity(x) <= self.y_max for x in x_vals])]
            precomputed_x_ranges.append(valid_x)

        # Create graphs and labels for each complexity
        for i, (complexity, label_text, valid_x) in enumerate(zip(self.complexities, self.labels, precomputed_x_ranges)):
            color = self.colors[i % len(self.colors)]

            # Plot the complexity curve with precomputed valid x-values
            graph = always_redraw(
                lambda f=complexity, v_x=valid_x, c=color: axes.plot(
                    f,
                    x_range=[0.1, min(k.get_value(), v_x[-1])],
                    color=c
                )
            )

            # Add label
            label = MathTex(label_text, color=color).scale(0.65)
            label.next_to(graph, UR, buff=0.1)

            # Updater for label position
            def update_label(obj, g=graph):
                obj.next_to(g, UR, buff=0.1)

            label.add_updater(update_label)

            graphs.add(graph)
            labels_group.add(label)

            self.add(graph, label)  # Add graph and label to the scene

        # Animate the graphs growing
        self.wait(1)
        self.play(k.animate.set_value(self.x_max), run_time=5)
        self.wait(2)


###### ----------------------------- spin qubits

from manim import *
import numpy as np

class ElectronSpinObject(VGroup):
    def __init__(self, position=ORIGIN, arrow_color=RED, arrow_direction="clockwise", **kwargs):
        super().__init__(**kwargs)

        # Create the electron
        self.electron = Circle(radius=1, color=BLUE, fill_opacity=1).move_to(position)

        # Create the spin arrow's ellipse path
        ellipse = Ellipse(width=3, height=1, color=arrow_color, stroke_width=8).move_to(position)

        # Intersection angles
        def find_intersection_angles(a, b, r):
            angles = []
            for sign in [-1, 1]:
                discriminant = (r**2 - b**2) / (a**2 - b**2)
                if discriminant < 0:
                    continue
                cos_theta = np.sqrt(discriminant) * sign
                if -1 <= cos_theta <= 1:
                    theta = np.arccos(cos_theta)
                    angles.append(theta)
            return sorted(angles)

        a, b = ellipse.width / 2, ellipse.height / 2
        intersection_angles = find_intersection_angles(a, b, self.electron.radius)

        # Arrow parts
        part1 = VMobject()
        part2_base = VMobject()
        part3 = VMobject()

        if arrow_direction == "clockwise":
            part1.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(-PI / 2 + 0.15, intersection_angles[0], 50)
            ])
            part2_base.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(intersection_angles[0], intersection_angles[1], 50)
            ])
            part3.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(intersection_angles[1], 3 * PI / 2 - 0.15, 50)
            ])
        else:
            part1.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(3 * PI / 2 - 0.15, intersection_angles[1], 50)
            ])
            part2_base.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(intersection_angles[1], intersection_angles[0], 50)
            ])
            part3.set_points_smoothly([
                ellipse.point_at_angle(angle)
                for angle in np.linspace(intersection_angles[0], -PI / 2 + 0.15, 50)
            ])

        part1.set_color(arrow_color).set_stroke(width=8)
        part2 = DashedVMobject(part2_base, num_dashes=15).set_color(arrow_color).set_stroke(width=8)
        part3.set_color(arrow_color).set_stroke(width=8)

        # Add arrowhead
        arrow_tip = ArrowTriangleFilledTip(color=arrow_color)
        arrow_tip.scale(1).move_to(part3.get_end())
        arrow_tip.rotate(PI if arrow_direction == "clockwise" else 0)
        combined_arrow = VGroup(part1, part2, part3, arrow_tip)

        # Store the components but don't add them to the scene initially
        self.components = VGroup(self.electron, combined_arrow)

    def add_to_scene(self, scene):
        scene.add(self.components)

    def animate_electron(self,scene):
        scene.play(GrowFromCenter(self.electron))

    def animate_spin(self, scene):
        parts = self.components[1].submobjects  # Access combined_arrow parts
        arrow_tip = parts[3]
        scene.play(Create(parts[0]), run_time=1)
        scene.play(Create(parts[1]))
        scene.play(Create(parts[2]), run_time=1)
        scene.play(Create(arrow_tip))


        # Usage of the custom animation in spin_flip
    def spin_flip(self, scene, new_color):
        # Create and apply the custom SpinFlip animation
        scene.play(
            SpinFlip(
                self.components[1],
                angle=PI,
                axis=UP,
                about_point=self.components[0].get_center(),
                new_color=new_color,
                run_time=2
            )
        )

class SpinFlip(Animation):
    def __init__(self, mobject, angle, axis, about_point, new_color, **kwargs):
        super().__init__(mobject, **kwargs)
        self.angle = angle
        self.axis = axis
        self.about_point = about_point
        self.new_color = new_color
        self.start_angle = 0  # To track the initial rotation state
        self.start_color = mobject.get_color()  # Store the initial color

    def begin(self):
        super().begin()
        self.start_angle = 0  # Reset the start angle to ensure correct rotation tracking

    def interpolate_mobject(self, alpha):
        # Compute the incremental rotation based on alpha
        incremental_angle = alpha * self.angle
        self.mobject.rotate(incremental_angle - self.start_angle, axis=self.axis, about_point=self.about_point)
        self.start_angle = incremental_angle

        # Interpolate the color change smoothly without jumping to white
        current_color = interpolate_color(self.start_color, self.new_color, alpha)
        self.mobject.set_color(current_color)


### -------------------------------------------------------- bloch sphere
from manim import *
import numpy as np

from manim import *

class BlochSphere(VGroup):
    def __init__(self, radius=2, label_type="cartesian", **kwargs):
        super().__init__(**kwargs)

        # Create the Bloch sphere
        self.sphere = Sphere(radius=radius, color=BLUE, fill_opacity=0.3)

        # Use Manim’s built-in Arrow3D for the axes
        self.x_axis = Arrow3D(start=ORIGIN, end=radius * RIGHT, color=WHITE)
        self.y_axis = Arrow3D(start=ORIGIN, end=radius * UP, color=WHITE)
        self.z_axis = Arrow3D(start=ORIGIN, end=radius * OUT, color=WHITE)
        self.nz_axis = Arrow3D(start=ORIGIN, end=-radius * OUT, color=WHITE)

        # Correct label rotations to match the 3D camera perspective
        if label_type == "cartesian":
            self.axes_labels = VGroup(
                Text("X", color=RED).rotate(-PI / 2, axis=UP).rotate(-PI / 4, axis=OUT).next_to(self.x_axis.get_end(), RIGHT),
                Text("Y", color=GREEN).rotate(-PI / 2, axis=OUT).rotate(-PI / 4, axis=RIGHT).next_to(self.y_axis.get_end(), UP),
                Text("Z", color=YELLOW).rotate(-PI / 2, axis=RIGHT).next_to(self.z_axis.get_end(), OUT)
            )
        elif label_type == "qubit":
            self.axes_labels = VGroup(
                MathTex(r"|+\rangle", color=WHITE).rotate(PI/2,RIGHT).rotate(3*PI/4,OUT).next_to(self.x_axis.get_end(), RIGHT, buff=1),
                MathTex(r"|0\rangle", color=WHITE).rotate(PI/2,RIGHT).rotate(3*PI/4,OUT).next_to(self.z_axis.get_end(), OUT),
                MathTex(r"|1\rangle", color=WHITE).rotate(PI/2,RIGHT).rotate(3*PI/4,OUT).next_to(self.nz_axis.get_end(), -OUT),
                MathTex(r"|+i\rangle", color=WHITE).rotate(PI/2,RIGHT).rotate(3*PI/4,OUT).next_to(self.y_axis.get_end(), UP, buff=1),
            )

        # Add components to the group
        self.add(self.sphere, self.x_axis, self.y_axis, self.z_axis, self.nz_axis, self.axes_labels)

    def add_state_vector(self, direction=RIGHT + UP + OUT, color=PURPLE):
        """Adds a state vector to the Bloch Sphere."""
        direction_normalized = direction / np.linalg.norm(direction)
        direction_on_sphere = direction_normalized * (self.sphere.radius - 0.05)

        # Use Manim's Arrow3D for the state vector
        self.state_vector = Arrow3D(start=ORIGIN, end=direction_on_sphere, color=color)
        self.add(self.state_vector)

    def initialize(self):
        """Sets the state vector to the |0⟩ state."""
        if hasattr(self, "state_vector"):
            self.remove(self.state_vector)
        self.add_state_vector(direction=OUT, color=PURPLE)

    def not_gate(self, scene):
        """Applies a NOT gate, flipping the state vector 180 degrees about the Y axis."""
        if hasattr(self, "state_vector"):
            scene.play(Rotate(self.state_vector, angle=PI, axis=RIGHT, about_point=ORIGIN, run_time=2))

    def hadamard_gate(self, scene):
        """Applies a Hadamard gate, rotating the state vector 90 degrees about the Y axis."""
        if hasattr(self, "state_vector"):
            scene.play(Rotate(self.state_vector, angle=PI / 2, axis=UP, about_point=ORIGIN, run_time=2))

    def z_rotation(self, scene, angle=PI / 2):
        """Rotates the state vector about the Z axis by the given angle."""
        if hasattr(self, "state_vector"):
            scene.play(Rotate(self.state_vector, angle=angle, axis=OUT, about_point=ORIGIN, run_time=2))


class MajoranaParticle(VGroup):
    def __init__(self, amplitude=1, width=1, resolution=100, color=RED, **kwargs):
        """
        Creates a visualization of a Majorana particle as a Gaussian wavefunction with a filled region.

        Parameters:
        - amplitude (float): Peak height of the Gaussian.
        - width (float): Initial spread of the wavefunction.
        - resolution (int): Number of points used to draw the wavefunction.
        - color (Color): Color of the wavefunction and fill.
        """
        super().__init__(**kwargs)
        self.initial_amplitude = amplitude
        self.initial_width = width
        self.resolution = resolution
        self.color = color  # Store the color

        # Track width changes
        self.width_tracker = ValueTracker(width)

        # Create the wavefunction and shaded area
        self.wavefunction = VMobject(stroke_width=3)
        self.fill_region = VMobject(fill_opacity=0.5, stroke_width=0)

        self.update_wavefunction()
        self.add(self.fill_region, self.wavefunction)  # Add fill before wavefunction for proper layering

    def gaussian_wave(self, x, width):
        """Defines a normalized Gaussian wavefunction."""
        norm_factor = 1 / (width * np.sqrt(2 * PI))  # Adjust height as width changes
        return self.initial_amplitude * norm_factor * np.exp(- (x**2) / (2 * width**2))

    def update_wavefunction(self):
        """Updates the wavefunction and its fill region based on the current width."""
        width = self.width_tracker.get_value()
        amplitude = self.initial_amplitude * (self.initial_width / width)  # Adjust height inversely with width

        x_values = np.linspace(-3 * width, 3 * width, self.resolution)
        wave_points = [np.array([x, amplitude * np.exp(- (x**2) / (2 * width**2)), 0]) for x in x_values]

        # Create shaded region by connecting the wavefunction to the x-axis
        fill_points = [np.array([x_values[0], 0, 0])] + wave_points + [np.array([x_values[-1], 0, 0])]

        self.wavefunction.set_points_smoothly(wave_points).set_color(self.color)
        self.fill_region.set_points_as_corners(fill_points).set_fill(self.color, opacity=0.5)

    def animate_wavefunction(self, scene, min_width=0.5, max_width=1.5, run_time=3):
        """
        Adds an animation where the wavefunction smoothly compresses and stretches.

        Parameters:
        - scene (Scene): The Manim scene to animate in.
        - min_width (float): Minimum width of the wavefunction during compression.
        - max_width (float): Maximum width during stretching.
        - run_time (float): Duration of the animation.
        """
        self.width_tracker.set_value(self.initial_width)  # Reset width
        self.wavefunction.add_updater(lambda m: self.update_wavefunction())
        self.fill_region.add_updater(lambda m: self.update_wavefunction())

        # Animate width oscillating between min and max values
        scene.play(
            self.width_tracker.animate.set_value(min_width),
            run_time=run_time / 2,
            rate_func=smooth
        )
        scene.play(
            self.width_tracker.animate.set_value(max_width),
            run_time=run_time / 2,
            rate_func=smooth
        )

        self.wavefunction.clear_updaters()  # Stop updating after animation
        self.fill_region.clear_updaters()


ImportError: cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.11/dist-packages/numpy/_core/umath.py)

## code

In [ ]:
from google.colab import drive;
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from manim import *

class SchroEqScroll(Scene):
    def construct(self):
        # Schrödinger equation with expanded Hamiltonian
        equation = MathTex(
            r"\left[ -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + V(x) \right] \psi(x) = E \psi(x)",
            font_size=42
        )

        # Position to the right side of screen
        equation.to_edge(RIGHT).shift(DOWN * 0.5+LEFT)

        # Animate scroll-in effect from below
        self.play(FadeIn(equation, shift=0.5 * DOWN), run_time=2)

        # Keep it on screen
        self.wait(4)


In [ ]:
%%manim -qm -v WARNING SchroEqScroll

Manim Community v0.19.0

In [ ]:
from manim import *

class SimpleVectorInterpretation(Scene):
    def construct(self):
        # Schrödinger equation on the left
        eq = MathTex(
            r"\left[ -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + V(x) \right] \psi(x) = E \psi(x)",
            font_size=36
        ).to_edge(LEFT).shift(DOWN * 0.5+RIGHT)

        # Axes on the right
        axes = Axes(
            x_range=[-2, 2],
            y_range=[-2, 2],
            x_length=3,
            y_length=3,
            axis_config={"include_tip": True}
        ).to_edge(RIGHT).shift(DOWN * 0.2+LEFT)

        # Vectors
        origin = axes.c2p(0, 0)
        vec1_end = axes.c2p(1, 1)
        vec2_end = axes.c2p(-1, 1)

        v1 = Arrow(start=origin, end=vec1_end, buff=0, color=BLUE)
        v2 = Arrow(start=origin, end=vec2_end, buff=0, color=RED)

        # Labels
        label_v1 = MathTex(r"\vec{v}_1", color=BLUE).next_to(vec1_end, UR)
        label_v2 = MathTex(r"\vec{v}_2", color=RED).next_to(vec2_end, UL)

        # Curved arrow from equation to graph
        arrow = CurvedArrow(eq.get_right()+0.25*RIGHT, axes.get_left()+0.25*LEFT, angle=-PI / 4)

        # Animate
        self.play(FadeIn(eq))
        self.wait(1)
        self.play(Create(arrow), run_time=1)
        self.play(Create(axes))
        self.play(Create(v1), FadeIn(label_v1))
        self.play(Create(v2), FadeIn(label_v2))
        self.wait(3)


In [ ]:
%%manim -qm -v WARNING SimpleVectorInterpretation

Manim Community v0.19.0

In [ ]:
from manim import *

class ComplexExponential(Scene):
    def construct(self):
        # Create the complex plane
        plane = NumberPlane(
            x_range=[-1.5, 1.5],
            y_range=[-1.5, 1.5],
            x_length=4,
            y_length=4,
            axis_config={"include_tip": True}
        )
        plane.add_coordinates()
        self.play(Create(plane), run_time=1.5)

        # Rotating vector and dot
        dot = Dot(plane.c2p(1, 0), color=YELLOW)
        vector = always_redraw(lambda: Arrow(
            start=plane.c2p(0, 0),
            end=dot.get_center(),
            buff=0,
            color=YELLOW
        ))

        # Label e^{iθ}
        label = always_redraw(lambda: MathTex(r"e^{i\theta}", color=YELLOW).scale(0.75).next_to(dot, UR, buff=0.2))

        # Add trailing path
        trail = TracedPath(dot.get_center, stroke_color=YELLOW, stroke_opacity=1)

        self.add(vector, dot, label, trail)

        # Animate rotation
        self.play(Rotate(dot, angle=2 * PI, about_point=plane.c2p(0, 0), run_time=6, rate_func=linear))
        self.wait(1)


In [ ]:
%%manim -qm -v WARNING ComplexExponential

Manim Community v0.19.0

In [ ]:
from manim import *

class LinearAlgebraStep(Scene):
    def construct(self):
        # 1. Blue vector on left (no axes)
        input_vec = Arrow(start=ORIGIN, end=2*RIGHT + UP, buff=0, color=BLUE)
        input_label = MathTex(r"\vec{v}", color=BLUE).next_to(input_vec.get_end(), UP)
        vec_group = VGroup(input_vec, input_label).to_edge(LEFT).shift(RIGHT)

        # 2. Matrix in center
        matrix_tex = MathTex(r"\begin{bmatrix} 2 & 1 \\ 0 & 1 \end{bmatrix}").scale(1.2)
        matrix_label = MathTex("T").next_to(matrix_tex, UP)

        # 3. Axes on right with transformed vector
        axes = Axes(
            x_range=[-1, 4],
            y_range=[-1, 3],
            x_length=3,
            y_length=3,
            axis_config={"include_tip": True}
        )
        transformed_vec = Arrow(
            start=axes.c2p(0, 0),
            end=axes.c2p(3, 1),
            buff=0,
            color=GREEN
        )
        output_label = MathTex(r"\mathbf{T}(\vec{v})", color=GREEN).scale(0.8).next_to(transformed_vec.get_end(), UR)
        output_group = VGroup(axes, transformed_vec, output_label).to_edge(RIGHT)

        # Animate all
        self.play(FadeIn(vec_group))
        self.play(FadeIn(matrix_tex))
        self.play(FadeIn(matrix_label))
        self.play(Create(axes))
        self.play(GrowArrow(transformed_vec), FadeIn(output_label))
        self.wait(2)


In [ ]:
%%manim -qm -v WARNING LinearAlgebraStep

Manim Community v0.19.0

In [ ]:
from manim import *

class VectorIntro(Scene):
    def construct(self):
        # Step 1: Axes on the left
        axes = Axes(
            x_range=[0, 4],
            y_range=[0, 3],
            x_length=3,
            y_length=2.25,
            axis_config={"include_tip": True}
        ).to_edge(LEFT).shift(RIGHT)

        self.play(Create(axes))

        # Step 2: 2D vector display on the right
        vec_tex = MathTex(r"\vec{v} = \begin{bmatrix} 2 \\ 1 \end{bmatrix}")
        vec_tex.to_edge(RIGHT).shift(LEFT)
        self.play(Write(vec_tex))

        # Step 3: Draw vector on graph
        origin = axes.c2p(0, 0)
        vec_end = axes.c2p(2, 1)
        arrow = Arrow(start=origin, end=vec_end, buff=0, color=BLUE)
        self.play(GrowArrow(arrow))

        # Step 4: Copy arrow, slide to right, align with vector text, rotate flat
        arrow_copy = arrow.copy().clear_updaters()
        self.play(arrow_copy.animate.move_to(ORIGIN + RIGHT*1+UP*2))

        vec_tex_2 = vec_tex.copy().next_to(arrow_copy, DOWN).shift(DOWN*1.75)
        self.play(TransformFromCopy(vec_tex, vec_tex_2))

        # Rotate arrow flat
        # Rotate arrow_copy to lie flat
        angle = np.arctan(0.5)  # angle between (2, 1) and x-axis
        self.play(Rotate(arrow_copy, angle=-angle, about_point=arrow_copy.get_start()))


        # Step 5: Measure magnitude with a number line
        number_line = NumberLine(
            x_range=[0, 3],
            length=2.5,
            include_tip=True,
            include_numbers=True,
            color=GREY
        ).next_to(arrow_copy, DOWN*2)
        self.play(Create(number_line))

        # Step 6: Show direction (angle)
        # Arc from horizontal to vector angle
        angle_arc = Arc(
            start_angle=0,
            angle=np.arctan(0.5),
            radius=0.25,
            color=RED
        ).move_arc_center_to(origin)

        theta_label = MathTex(r"\theta").move_to(angle_arc.get_center() + 0.35*UP + 0.15*RIGHT)
        self.play(Create(angle_arc), FadeIn(theta_label))

        self.wait(2)


In [ ]:
%%manim -qm -v WARNING VectorIntro

Manim Community v0.19.0

In [ ]:
from manim import *

class ColumnVectorLabels(Scene):
    def construct(self):
        # ValueTrackers for 3 vectors (x, y, z)
        trackers = [
            [ValueTracker(1), ValueTracker(1), ValueTracker(0)],
            [ValueTracker(0), ValueTracker(2), ValueTracker(1)],
            [ValueTracker(1), ValueTracker(-1), ValueTracker(2)],
        ]

        colors = [RED, GREEN, BLUE]
        labels = VGroup()

        for i, (tx, ty, tz) in enumerate(trackers):
            def make_label(t=tx, u=ty, v=tz, c=colors[i], index=i):
                return always_redraw(lambda:
                    MathTex(
                        r"\begin{bmatrix}" +
                        f"{t.get_value():.1f} \\\\ {u.get_value():.1f} \\\\ {v.get_value():.1f}" +
                        r"\end{bmatrix}",
                        color=c
                    ).scale(1).move_to(RIGHT * (index - 1) * 2)
                )
            labels.add(make_label())

        self.add(labels)
        self.wait(1)

        # Animate changes
        self.play(
            trackers[0][0].animate.set_value(2),
            trackers[0][1].animate.set_value(-1),
            trackers[0][2].animate.set_value(1),
            trackers[1][0].animate.set_value(-2),
            trackers[1][1].animate.set_value(1),
            trackers[1][2].animate.set_value(-1),
            trackers[2][0].animate.set_value(0),
            trackers[2][1].animate.set_value(0),
            trackers[2][2].animate.set_value(3),
            run_time=4
        )
        self.wait(2)


In [ ]:
from manim import *

class VectorSpace3D(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=-45 * DEGREES)

        axes = ThreeDAxes(
            x_range=[-3, 3],
            y_range=[-3, 3],
            z_range=[-3, 3],
            x_length=4,
            y_length=4,
            z_length=4
        )
        self.add(axes)

        # Matching ValueTrackers (same values as in 2D scene)
        trackers = [
            [ValueTracker(1), ValueTracker(1), ValueTracker(0)],
            [ValueTracker(0), ValueTracker(2), ValueTracker(1)],
            [ValueTracker(1), ValueTracker(-1), ValueTracker(2)],
        ]

        colors = [RED, GREEN, BLUE]
        vectors = VGroup()

        for i, (tx, ty, tz) in enumerate(trackers):
            arrow = always_redraw(lambda t=tx, u=ty, v=tz, c=colors[i]:
                Arrow3D(start=ORIGIN, end=[t.get_value(), u.get_value(), v.get_value()], color=c)
            )
            vectors.add(arrow)

        self.add(vectors)
        self.wait(1)

        self.play(
            trackers[0][0].animate.set_value(2),
            trackers[0][1].animate.set_value(-1),
            trackers[0][2].animate.set_value(1),
            trackers[1][0].animate.set_value(-2),
            trackers[1][1].animate.set_value(1),
            trackers[1][2].animate.set_value(-1),
            trackers[2][0].animate.set_value(0),
            trackers[2][1].animate.set_value(0),
            trackers[2][2].animate.set_value(3),
            run_time=4
        )
        self.wait(2)


In [ ]:
%%manim -qm -v WARNING VectorSpace3D

Manim Community v0.19.0

In [ ]:
%%manim -qm -v WARNING ColumnVectorLabels

Manim Community v0.19.0

In [ ]:
from manim import *

class VectorAdditionColumnNotation(Scene):
    def construct(self):
        # Step 1: 2D axes
        axes = Axes(
            x_range=[0, 2],
            y_range=[0, 2],
            x_length=4,
            y_length=4,
            axis_config={"include_tip": True}
        ).to_edge(LEFT).shift(RIGHT * 1.5)
        self.play(Create(axes))

        origin = axes.c2p(0, 0)
        e1_end = axes.c2p(1, 0)
        e2_end = axes.c2p(1, 1)

        # Step 2: Vectors
        e1 = Arrow(start=origin, end=e1_end, buff=0, color=BLUE)
        e2 = Arrow(start=e1_end, end=e2_end, buff=0, color=GREEN)
        v = Arrow(start=origin, end=e2_end, buff=0, color=YELLOW)

        # Step 3: Labels as column vectors
        label_e1 = MathTex(r"\begin{bmatrix} 1 \\ 0 \end{bmatrix}", color=BLUE).scale(0.7).next_to(e1, DOWN)
        label_e2 = MathTex(r"\begin{bmatrix} 0 \\ 1 \end{bmatrix}", color=GREEN).scale(0.7).next_to(e2, RIGHT)
        label_v = MathTex(r"\begin{bmatrix} 1 \\ 1 \end{bmatrix}", color=YELLOW).scale(0.7).next_to(v, LEFT)

        # Step 4: Equation at top
        equation = MathTex(
            r"\vec{v} = \begin{bmatrix} 1 \\ 0 \end{bmatrix} + \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 1 \\ 1 \end{bmatrix}"
        ).to_edge(UP)

        # Step 5: Animate in sequence
        self.play(Write(equation))
        self.wait(0.5)
        self.play(GrowArrow(e1), FadeIn(label_e1))
        self.play(GrowArrow(e2), FadeIn(label_e2))
        self.play(GrowArrow(v), FadeIn(label_v))
        self.wait(4)


In [ ]:
%%manim -qm -v WARNING VectorAdditionColumnNotation

Manim Community v0.19.0

In [ ]:
from manim import *

class BasisLinearCombo(Scene):
    def construct(self):
        # Axes
        axes = Axes(
            x_range=[-1, 4],
            y_range=[-1, 4],
            x_length=5,
            y_length=5,
            axis_config={"include_tip": True}
        ).to_edge(LEFT).shift(RIGHT * 1.5)
        self.play(Create(axes))

        # ValueTrackers for scalars
        a = ValueTracker(1)
        b = ValueTracker(1)

        # Scaled vectors: a*[1,0], b*[0,1]
        vec_a = always_redraw(lambda:
            Arrow(
                start=axes.c2p(0, 0),
                end=axes.c2p(a.get_value(), 0),
                buff=0,
                color=BLUE
            )
        )

        vec_b = always_redraw(lambda:
            Arrow(
                start=axes.c2p(a.get_value(), 0),
                end=axes.c2p(a.get_value(), b.get_value()),
                buff=0,
                color=GREEN
            )
        )

        vec_sum = always_redraw(lambda:
            Arrow(
                start=axes.c2p(0, 0),
                end=axes.c2p(a.get_value(), b.get_value()),
                buff=0,
                color=YELLOW
            )
        )

        # Equation at top
        equation = always_redraw(lambda:
            MathTex(
                r"\vec{v} = " +
                f"{a.get_value():.1f}" + r"\begin{bmatrix} 1 \\ 0 \end{bmatrix} + " +
                f"{b.get_value():.1f}" + r"\begin{bmatrix} 0 \\ 1 \end{bmatrix} = " +
                r"\begin{bmatrix}" + f"{a.get_value():.1f} \\\\ {b.get_value():.1f}" + r"\end{bmatrix}"
            ).to_edge(UP)
        )

        # Add everything
        self.add(vec_a, vec_b, vec_sum, equation)
        self.wait(1)

        # Animate changing the scalars
        self.play(a.animate.set_value(2), b.animate.set_value(3), run_time=3)
        self.play(a.animate.set_value(-1), b.animate.set_value(2), run_time=3)
        self.wait(2)


In [ ]:
%%manim -qm -v WARNING BasisLinearCombo

Manim Community v0.19.0

In [ ]:
from manim import *

class ColumnVector3DLabels(Scene):
    def construct(self):
        # ValueTrackers for x, y, z
        x = ValueTracker(1)
        y = ValueTracker(1)
        z = ValueTracker(1)

        # Dynamic equation at the top
        equation = always_redraw(lambda:
            MathTex(
                r"\vec{r} = " +
                f"{x.get_value():.1f}" + r"\begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix} + " +
                f"{y.get_value():.1f}" + r"\begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix} + " +
                f"{z.get_value():.1f}" + r"\begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} = " +
                r"\begin{bmatrix}" + f"{x.get_value():.1f} \\\\ {y.get_value():.1f} \\\\ {z.get_value():.1f}" + r"\end{bmatrix}"
            ).scale(0.9).to_edge(UP)
        )

        self.add(equation)
        self.wait(1)

        # Animate changing the vector components
        self.play(x.animate.set_value(2), y.animate.set_value(1), z.animate.set_value(3), run_time=3)
        self.play(x.animate.set_value(0), y.animate.set_value(-2), z.animate.set_value(1), run_time=3)
        self.wait(2)


In [ ]:
from manim import *

class VectorBasis3D(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=-45 * DEGREES)

        # 3D Axes
        axes = ThreeDAxes(
            x_range=[-3, 3],
            y_range=[-3, 3],
            z_range=[-1, 4],
            x_length=5,
            y_length=5,
            z_length=4
        )
        self.add(axes)

        # ValueTrackers for components
        x = ValueTracker(1)
        y = ValueTracker(1)
        z = ValueTracker(1)

        # Basis vectors
        vec_x = always_redraw(lambda:
            Arrow3D(
                start=ORIGIN,
                end=[x.get_value(), 0, 0],
                color=RED
            )
        )
        vec_y = always_redraw(lambda:
            Arrow3D(
                start=[x.get_value(), 0, 0],
                end=[x.get_value(), y.get_value(), 0],
                color=GREEN
            )
        )
        vec_z = always_redraw(lambda:
            Arrow3D(
                start=[x.get_value(), y.get_value(), 0],
                end=[x.get_value(), y.get_value(), z.get_value()],
                color=BLUE
            )
        )
        vec_total = always_redraw(lambda:
            Arrow3D(
                start=ORIGIN,
                end=[x.get_value(), y.get_value(), z.get_value()],
                color=YELLOW
            )
        )

        self.add(vec_x, vec_y, vec_z, vec_total)
        self.wait(1)

        # Animate changes (same as in 2D)
        self.play(x.animate.set_value(2), y.animate.set_value(1), z.animate.set_value(3), run_time=3)
        self.play(x.animate.set_value(0), y.animate.set_value(-2), z.animate.set_value(1), run_time=3)
        self.wait(2)


In [ ]:
%%manim -qm -v WARNING ColumnVector3DLabels

Manim Community v0.19.0

In [ ]:
%%manim -qm -v WARNING VectorBasis3D

Manim Community v0.19.0

In [ ]:
from manim import *

class LinearTransformDemo(Scene):
    def construct(self):
        # Starting square
        square = Square(side_length=2, color=BLUE).move_to(LEFT * 3)
        self.play(Create(square))
        self.wait(1)

        # Transformation 1: Rotation
        rotation_angle = PI / 4  # 45 degrees
        matrix_rot = MathTex(
              r"\begin{bmatrix}"
              r"\tiny \cos(\pi/4) & \tiny -\sin(\pi/4) \\"
              r"\tiny \sin(\pi/4) & \tiny \cos(\pi/4)"
              r"\end{bmatrix}"
          ).scale(1).move_to(RIGHT * 3)


        label_rot = Text("Rotation", font_size=28).next_to(matrix_rot, UP)

        self.play(Write(label_rot), Write(matrix_rot))
        self.play(Rotate(square, angle=rotation_angle, about_point=square.get_center()))
        self.wait(1)
        self.play(FadeOut(label_rot), FadeOut(matrix_rot))

        # Transformation 2: Translation (represented here symbolically with a matrix-like form)
        matrix_trans = Matrix([
            ["1", "0", "dx"],
            ["0", "1", "dy"],
            ["0", "0", "1"]
        ], left_bracket="(", right_bracket=")").scale(0.75).move_to(RIGHT * 3)
        label_trans = Text("Translation", font_size=28).next_to(matrix_trans, UP)

        self.play(Write(label_trans), Write(matrix_trans))
        self.play(square.animate.shift(RIGHT + UP))
        self.wait(1)
        self.play(FadeOut(label_trans), FadeOut(matrix_trans))

        # Transformation 3: Shear
        shear_matrix = [[1, 0.5], [0, 1]]
        matrix_shear = Matrix([
            ["1", "0.5"],
            ["0", "1"]
        ], left_bracket="(", right_bracket=")").scale(0.75).move_to(RIGHT * 3)
        label_shear = Text("Shear", font_size=28).next_to(matrix_shear, UP)

        self.play(Write(label_shear), Write(matrix_shear))
        self.play(square.animate.apply_matrix(shear_matrix))
        self.wait(2)


In [ ]:
%%manim -pqh -v WARNING LinearTransformDemo

Manim Community v0.19.0

In [ ]:
from manim import *

class QHOProbability(Scene):
    def construct(self):
        # Step 1: Show the ground state wavefunction
        wave_eq = MathTex(
            r"\psi_0(x) = \left( \frac{m\omega}{\pi\hbar} \right)^{1/4} e^{-\frac{m\omega x^2}{2\hbar}}"
        ).scale(.75).shift(LEFT*3.75+UP)
        self.play(Write(wave_eq))
        self.wait(2)

        # Step 2: Show the probability density expression
        prob_eq = MathTex(
            r"|\psi_0(x)|^2 = \left( \frac{m\omega}{\pi\hbar} \right)^{1/2} e^{-\frac{m\omega x^2}{\hbar}}"
        ).next_to(wave_eq, DOWN, buff=1)

        self.play(TransformFromCopy(wave_eq, prob_eq))
        self.wait(2)

        # Step 3: Graph the probability distribution
        axes = Axes(
            x_range=[-4, 4],
            y_range=[0, 1.1],
            x_length=10,
            y_length=3,
            axis_config={"include_tip": False},
        ).next_to(prob_eq, RIGHT, buff=0).scale(.75).shift(LEFT+UP*0.5)

        # Axis label
        y_label = axes.get_y_axis_label(r"|\psi_0(x)|^2", direction=UP, buff=0.4)

        # Gaussian probability distribution: just set constants = 1 for simplicity
        graph = axes.plot(lambda x: np.exp(-x**2), color=YELLOW)

        self.play(Create(axes), Write(y_label))
        self.play(Create(graph))
        self.wait(3)


In [ ]:
%%manim -pqh -v WARNING QHOProbability
config.media_embed=True

Manim Community v0.19.0

In [ ]:
from manim import *

class ComplexConjugateMultiplication(Scene):
    def construct(self):
        # Step 1: Show the complex number z
        z = MathTex("z = a + bi").to_edge(UP)
        self.play(Write(z))
        self.wait(1)

        # Step 2: Show the complex conjugate of z
        z_conj = MathTex(r"\overline{z} = a - bi").next_to(z, DOWN, buff=1)
        self.play(Write(z_conj))
        self.wait(1)

        # Step 3: Show the multiplication of z and its conjugate
        multiply = MathTex(r"z \times \overline{z} = (a + bi)(a - bi)").next_to(z_conj, DOWN, buff=1)
        self.play(Write(multiply))
        self.wait(1)

        # Step 4: Expand and simplify
        expanded = MathTex(r"= a^2 - abi + abi - b^2 i^2").next_to(multiply, DOWN, buff=0.8)
        simplified = MathTex(r"= a^2 + b^2").next_to(expanded, DOWN, buff=0.8)

        self.play(Write(expanded))
        self.wait(1)
        self.play(Write(simplified))
        self.wait(2)


In [ ]:
%%manim -pqh -v WARNING ComplexConjugateMultiplication

Manim Community v0.19.0